In [83]:
import pandas as pd
from scipy import stats
import numpy as np
from pandas.api.types import CategoricalDtype

In [84]:
pl0 = pd.read_csv("Kraftwerksliste_CSV.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=9)
bm = pd.read_csv("block_plant_mapper.csv", error_bad_lines=False, engine="python")

In [85]:
pl0.rename(columns={ pl0.columns[10]: "Energieträger_old", pl0.columns[13]: "Energieträger"}, inplace=True)

In [86]:
#bm

In [87]:
#bm
pl1 = pl0.loc[np.logical_or(pl0.Energieträger == "Kernenergie", np.logical_or(pl0.Energieträger == "Erdgas", np.logical_or(pl0.Energieträger == "Steinkohle", pl0.Energieträger == "Braunkohle"))), :]

In [88]:
plt = pl1.dropna(subset=["Kraftwerksnummer Bundesnetzagentur"])

In [89]:
plt["Energieträger"] = plt["Energieträger"].astype('category')
plt["Energieträger"] = plt["Energieträger"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas"], ordered=True)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [90]:
# plt.dtypes

In [91]:
pl2 = plt.rename(columns={ pl1.columns[0]: "BlockID", pl1.columns[3]: "PLZ", pl1.columns[4]: "Ort", pl1.columns[8]: "Inbetriebnahme", pl1.columns[9]: "Status", pl1.columns[15]: "KWK", pl1.columns[16]: "Nettoleistung"})
#pl3 = pl2.rename(columns={ pl1.columns[1]: "BlockID" })

In [92]:
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('.','')
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(',','.')
pl2.loc[pl2.BlockID == 'BNA0861a', 'Inbetriebnahme'] = 2012
pl2.loc[pl2.BlockID == 'BNA1334', 'Inbetriebnahme'] = 2002
pl2.loc[pl2.BlockID == 'BNA1141', 'Inbetriebnahme'] = 1970
pl2.loc[pl2.BlockID == 'BNA0418', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.BlockID == 'BNA1499', 'Inbetriebnahme'] = 1951
pl2.loc[pl2.BlockID == 'BNA1502', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.BlockID == 'BNA1500', 'Inbetriebnahme'] = 1990
pl2.loc[pl2.BlockID == 'BNA1498', 'Inbetriebnahme'] = 1953
pl2.loc[pl2.BlockID == 'BNA1260', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.BlockID == 'BNA1056', 'Inbetriebnahme'] = 2006


#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('\n','')
#pl2['Nettoleistung'] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(';','.')

In [93]:
#pl2[] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
pl2['Inbetriebnahme'] = pl2['Inbetriebnahme'].apply(pd.to_numeric)
pl2['Nettoleistung'] = pl2['Nettoleistung'].apply(pd.to_numeric)

In [94]:
#bm[pl2.duplicated(['BlockID'], keep=False)].sort_values("BlockID", ascending=False)

In [95]:
pl2[pl2.duplicated(['BlockID'], keep=False)].sort_values("BlockID", ascending=False)

,BlockID,Unternehmen,Kraftwerksname,PLZ,Ort,Straße und Hausnummer (Standort Kraftwerk),Bundesland,Blockname,Inbetriebnahme,Status,Energieträger_old,"Spezifizierung ""Mehrere Energieträger"" und ""Sonstige Energieträger"" - Hauptbrennstoff","Spezifizierung ""Mehrere Energieträger"" - Zusatz- / Ersatzbrennstoffe",Energieträger,Vergütungsfähig nach EEG (ja/nein),KWK,Nettoleistung,Bezeichnung Verknüpfungspunkt (Schaltanlage) mit dem Stromnetz der Allgemeinen Versorgung gemäß Netzbetreiber,Netz- oder Umspannebene des Anschlusses in kV,Name Stromnetzbetreiber


In [96]:
cols = [10, 11,12,14,17,18,19]
pl3 = pl2.drop(pl2.columns[cols],axis=1)

In [97]:
#

In [98]:
# Status: endgültig stillgelegt, vorläufig stillgelegt, in Betrieb, Sicherheitsbereitschaft, Sonderfall, 
# Stillegung: [Jahreszahl]
# StA: ja/ nein

In [99]:
newdf = pd.DataFrame()

In [100]:
def generate_decom_tuple(state):
    if state == "In Betrieb":
        return("in Betrieb", "0")
    elif "Endgültig Stillgelegt" in state:
        a = "stillgelegt"
        b = str(int(state[22:26]))
        return (a, b)
    elif "Vorläufig Stillgelegt" in state:
        a = "vorläufig stillgelegt"
        return (a, "0")
    else:
        return(state, "0")

In [101]:
def fix_company(company):
    cmp = company
    company_prefix, _ = company.split(" " , 1)
    # company_dict = {}
    company_dict = {"RWE": "RWE AG", "Vattenfall": "Vattenfall GmbH", "Uniper": "Uniper SE", "EnBW": "EnBW AG", "Steag": "Steag GmbH", "Nordzucker": "Nordzucker AG"}
    try:
        cmp = company_dict[company_prefix]
    except KeyError:
        return company
    return cmp

In [102]:
for index, row in pl3.iterrows():
    #if row['Status'] == "Endgültig Stillgelegt 2016 (mit StA)":
    # print(row)
    if row['KWK'] == "nein":
        row['KWK'] = "Nein" # workaround for BoA plants
    if row['KWK'] == "ja":
        row['KWK'] = "Ja"
    status, stilllegung = generate_decom_tuple(row['Status'])
    cmp = fix_company(row['Unternehmen'])
    row['Unternehmen'] = cmp
    row['Status'] = status
    row['Stilllegung'] = stilllegung
    newdf = newdf.append(row)
    

In [103]:
# newdf['Stilllegung'] = newdf['Stilllegung'].astype(int)
#newdf['Stilllegung'] = newdf['Stilllegung'].apply(lambda x: int(x) if x == x else None)

In [104]:
newdf['Stilllegung'] = newdf['Stilllegung'].replace("0", np.nan)

In [105]:
newdf.dtypes

BlockID                                        object
Blockname                                      object
Bundesland                                     object
Energieträger                                  object
Inbetriebnahme                                float64
KWK                                            object
Kraftwerksname                                 object
Nettoleistung                                 float64
Ort                                            object
PLZ                                            object
Status                                         object
Stilllegung                                    object
Straße und Hausnummer (Standort Kraftwerk)     object
Unternehmen                                    object
dtype: object

In [106]:
newdf["Status"] = newdf["Status"].astype('category')
newdf["Status"] = newdf["Status"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)

In [107]:
newdf["KWK"] = newdf["KWK"].astype('category')
newdf["KWK"] = newdf["KWK"].cat.set_categories(['Ja', 'Nein'], ordered=True)

In [108]:
atest = newdf.groupby("KWK")

In [109]:
atest.count()

,BlockID,Blockname,Bundesland,Energieträger,Inbetriebnahme,Kraftwerksname,Nettoleistung,Ort,PLZ,Status,Stilllegung,Straße und Hausnummer (Standort Kraftwerk),Unternehmen
KWK,,,,,,,,,,,,,
Ja,380,272,380,380,379,369,377,380,380,380,31,231,380
Nein,127,117,127,127,127,127,127,127,127,127,38,37,127


In [110]:
pl3 = newdf

In [111]:
atest = newdf.groupby(["Unternehmen"]).count()
atest.sort_values("Inbetriebnahme", inplace=True, ascending=False)

In [112]:
#print(atest)

In [113]:
atest = pl3.groupby(["Status"]).count()
# atest

In [114]:
# pl3.rename(columns={ pl3.columns[10]: "Energieträger"}, inplace=True)

In [115]:
bm[bm.duplicated(['BlockID'], keep=False)].sort_values("BlockID", ascending=False)

,KraftwerkID,BlockID


In [116]:
#bm2 = bm.drop_duplicates()

In [117]:
pl4 = bm.merge(pl3, how="outer", on="BlockID")

In [118]:
#pl4[pl4.duplicated(['BlockID'], keep=False)].sort_values("BlockID", ascending=False)

In [119]:
pl5 = pl4[pd.notnull(pl4['BlockID'])]
#pl5 = pl4
pl6 = pl5[pd.notnull(pl5['Nettoleistung'])]

In [120]:
# pl6

In [121]:
pl6['KraftwerkID'] = pl6['KraftwerkID'].str.replace('/', '_')

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [122]:
# pl7 = pl6.replace("/", "_")

In [123]:
#pl7 = pl6.apply(str.replace("/", "_"))

In [124]:
pl6.loc[pl6.Kraftwerksname == "KW Mittelsbüren"]

,KraftwerkID,BlockID,Blockname,Bundesland,Energieträger,Inbetriebnahme,KWK,Kraftwerksname,Nettoleistung,Ort,PLZ,Status,Stilllegung,Straße und Hausnummer (Standort Kraftwerk),Unternehmen
367,06-04-11_2001178_8_2,BNA1820,GuD MiBÜ,Bremen,Erdgas,2016.0,Nein,KW Mittelsbüren,444.5,Bremen,28237,in Betrieb,NaN,NaN,Gemeinschaftskraftwerk Bremen GmbH und Co. KG


In [125]:
pl6['Inbetriebnahme'] = pl6['Inbetriebnahme'].apply(pd.to_numeric)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [126]:
blocks = pl6.copy()
stammdaten = pl6.copy()

In [127]:
blocks.sort_values("Inbetriebnahme", inplace=True, ascending=False)
# blocks

In [128]:
plants = blocks.loc[:]
plants = plants.dropna(subset=["KraftwerkID"])

In [129]:
#plants_d = plants.loc[plants.duplicated("KraftwerkID", keep=False)]
# plants_d

In [130]:
#plants_c = plants_d.groupby(["KraftwerkID"]).sum()
#plants_count = plants_d.groupby(["KraftwerkID"]).count()

In [131]:
#plants_f = plants_c.reset_index()

In [132]:
#plants_count = plants_count.loc[:, ["BlockID"]]

In [133]:
#plants_count = plants_count.reset_index()
# plants_count 

In [134]:
#plants_f = plants_f.merge(plants_count, on="KraftwerkID", how="inner")
# plants_f

In [135]:
#plants[plants.duplicated(['KraftwerkID'], keep=False)].sort_values("BlockID", ascending=False)

In [136]:
plants["Energieträger"] = plants["Energieträger"].astype('category')
plants["Energieträger"] = plants["Energieträger"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas"], ordered=True)
plants["Status"] = plants["Status"].astype('category')
plants["Status"] = plants["Status"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)
plants["KWK"] = plants["KWK"].astype('category')
plants["KWK"] = plants["KWK"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants["Bundesland"] = plants["Bundesland"].astype('category')
plants["Bundesland"] = plants["Bundesland"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [137]:
p_grp = plants.groupby("KraftwerkID")

In [138]:
plants.groupby("Energieträger").max()

,KraftwerkID,BlockID,Inbetriebnahme,Nettoleistung,Ort,PLZ,Unternehmen
Energieträger,,,,,,,
Kernenergie,None,None,NaN,NaN,None,None,None
Braunkohle,18040,BNA1511a,2012.0,1060.0,Zülpich,9116,eins - energie in sachsen GmbH & Co. KG
Steinkohle,13-30-1010101,BNA1674,2015.0,875.0,Zolling,91052,swb Erzeugung GmbH & Co. KG
Erdgas,GAS-0002,BNA1820,2016.0,887.0,Zielitz,99087,envia THERM GmbH


In [139]:
plants_a = pd.DataFrame()
for plantid, group in p_grp:
    #if not plantid == "06-05-300-9046797":
    #    continue
    entry = {}
    #print(entry)
    entry["plantid"] = plantid
    # entry["BlockID"] = group
    try:
        entry["plantname"] = group["Kraftwerksname"].value_counts().index[0]
    except IndexError:
        entry["plantname"] = np.NaN
    entry["federalstate"] = group["Bundesland"].value_counts().index[0]
    entry["energysource"] = group["Energieträger"].min()
    try:
        entry["chp"] = group["KWK"].min()
    except TypeError:
        #print("aneror")
        #print(plantid)
        #print(group['KWK'].count())
        entry["chp"] = ""
    entry["latestexpanded"] = group["Inbetriebnahme"].max()
    entry["initialop"] = group["Inbetriebnahme"].min()
    entry["totalpower"] = group["Nettoleistung"].sum()
    entry["state"] = group["Status"].min()
    entry["blockcount"] = group["BlockID"].count()
    entry["company"] = group["Unternehmen"].value_counts().index[0]
    #print(entry)
    #break
    plants_a = plants_a.append(entry, ignore_index=True)

In [140]:
int_list = ['blockcount', 'initialop', 'latestexpanded']
for column in int_list:
    plants_a[column] = plants_a[column].astype(int)
#plant['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)

In [141]:
plants_a.dtypes

blockcount          int64
chp                object
company            object
energysource       object
federalstate       object
initialop           int64
latestexpanded      int64
plantid            object
plantname          object
state              object
totalpower        float64
dtype: object

In [142]:
plants_a["energysource"] = plants_a["energysource"].astype('category')
plants_a["energysource"] = plants_a["energysource"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas"], ordered=True)
plants_a["state"] = plants_a["state"].astype('category')
plants_a["state"] = plants_a["state"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)
plants_a["chp"] = plants_a["chp"].astype('category')
plants_a["chp"] = plants_a["chp"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants_a["federalstate"] = plants_a["federalstate"].astype('category')
plants_a["federalstate"] = plants_a["federalstate"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [143]:
plants_a.dtypes

blockcount           int64
chp               category
company             object
energysource      category
federalstate      category
initialop            int64
latestexpanded       int64
plantid             object
plantname           object
state             category
totalpower         float64
dtype: object

In [144]:
plants_a.loc[plants_a.plantid == "06-05-300-0326774"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
49,9,Ja,RWE AG,Braunkohle,Nordrhein-Westfalen,1963,2002,06-05-300-0326774,Niederaußem,in Betrieb,3656.0


In [145]:
plants_a.loc[plants_a.plantid == "06-00176010435"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
20,4,Ja,Uniper SE,Steinkohle,Hessen,1965,1992,06-00176010435,Staudinger,in Betrieb,1624.0


In [146]:
plants_a.loc[plants_a.plantid == "06-05-900-0865327"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
71,9,Nein,RWE AG,Steinkohle,Nordrhein-Westfalen,1973,1984,06-05-900-0865327,Gersteinwerk,in Betrieb,2011.0


In [147]:
#plants_a = plants.drop_duplicates(subset="KraftwerkID")

In [148]:
column_titles = ['plantid', 'plantname', 'federalstate','energysource', 'chp', 'latestexpanded', 'initialop', 'totalpower', 'state', 'blockcount', 'company']
plants_b = plants_a.reindex(columns=column_titles)

In [149]:
#plants_b

In [150]:
blocks.drop(['Ort', 'PLZ', 'Straße und Hausnummer (Standort Kraftwerk)', 'Blockname'], axis=1, inplace=True)

In [151]:
blocks[blocks.duplicated(['BlockID'], keep=False)].sort_values("BlockID", ascending=False)

,KraftwerkID,BlockID,Bundesland,Energieträger,Inbetriebnahme,KWK,Kraftwerksname,Nettoleistung,Status,Stilllegung,Unternehmen


In [152]:
plants_b.sort_values("initialop", ascending=False, inplace=True)

In [153]:
#drop_list = ['Blockname', 'KWK', 'Nettoleistung', 'Stilllegung', 'Straße und Hausnummer (Standort Kraftwerk)', 'PLZ', 'Ort', 'Nettoleistung']
#plants_g = plants_f.drop(drop_list, axis=1)

In [154]:
#plants_h = plants_g.rename(columns={"KraftwerkID": "plantid", "Gesamtleistung": "totalpower", "Blockzahl": "blockcount", "BlockID": "blockid", "Bundesland": "federalstate", "Energieträger": "energysource", "Inbetriebnahme": "latestexpanded", "Kraftwerksname": "plantname", "Status": "state", "Unternehmen": "company"})

In [155]:
plants_final = plants_b.loc[:]

In [156]:
plants_final.dtypes

plantid             object
plantname           object
federalstate      category
energysource      category
chp               category
latestexpanded       int64
initialop            int64
totalpower         float64
state             category
blockcount           int64
company             object
dtype: object

In [157]:
# plants_final

In [158]:
# stammdaten

In [159]:
# cols = [0,2,3,9,10,11,12]
drop_list = ['KraftwerkID', 'Blockname', 'Energieträger', 'Inbetriebnahme', 'KWK', 'Kraftwerksname', 'Nettoleistung', 'Status', 'Stilllegung', 'Unternehmen']
stammdaten.drop(drop_list, axis=1, inplace=True)

In [160]:
stammdaten.to_csv("stammdaten.csv", index=False)

In [161]:
plants_final.to_csv("plants.csv", index=False)

In [162]:
blocks.to_csv("blocks_3.csv", index=False)

In [163]:
stammdaten.to_csv("stammdaten_nh.csv", index=False, header=False)
blocks.to_csv("blocks_nh.csv", index=False, header=False)
plants_final.to_csv("plants_nh.csv", index=False, header=False)

In [164]:
#bm2.to_csv("bpm.csv", index=False)